# Hessian & batch executation

This example shows how to obtan hessian and how to perform batch executation. Since functorch functions (vmap, hessian, jacfwd, etc.) currently do not support the use of autograd.Function. Hessian & batch executation function are only supported by PyTorch backend backpropagation autograd method now.

# Initialization

In [ ]:
import tedq as qai
import torch
from functorch import hessian, jacfwd, vmap

# Define the quantum model

### Define the circuit with TeD-Q framework
#### (Remember, if you have multiple measurements, all the measurement results should has the same shape!)

In [ ]:
# Define quantum circuit
def circuitDef(params):
    qai.RX(params[0], qubits=[0])
    qai.RY(params[1], qubits=[0])
    return qai.expval(qai.PauliZ(qubits=[0]))

number_of_qubits = 1
parameter_shapes = [(2,)]

# Quantum circuit construction
circuit = qai.Circuit(circuitDef, number_of_qubits, parameter_shapes = parameter_shapes)

In [ ]:
# visualization of the quantum circuit
drawer = qai.matplotlib_drawer(circuit)
drawer.draw_circuit()

# Circuit compiled with pytorch backend

Gradient will obtain from backpropagation by default

### state vector propagation mode

In [ ]:
# my_compilecircuit = circuit.compilecircuit(backend="pytorch")

### tensor network contraction mode

#### Use CoTenGra

In [ ]:
# slicing_opts = {'target_size': 2**28}
# hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'progbar':True, 'minimize':'flops', 'parallel':True, 'slicing_opts':slicing_opts}
# import cotengra as ctg
# my_compilecircuit = circuit.compilecircuit(backend="pytorch", use_cotengra=ctg, hyper_opt = hyper_opt)

#### Use JDtensorPath (Suggested)
1. 'target_num_slices' is useful if you want to do the contraction in parallel, it will devide the tensor network into pieces and then calculat them in parallel
2. 'math_repeats' means how many times are going to run JDtensorPath to find a best contraction path
3. 'search_parallel' means to run the JDtensorPath in parallel, True means to use all the CPUs, integer number means to use that number of CPUs


In [ ]:
from jdtensorpath import JDOptTN as jdopttn
slicing_opts = {'target_size':2**28, 'repeats':500, 'target_num_slices':None, 'contract_parallel':False}
hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'search_parallel':True, 'slicing_opts':slicing_opts}
my_compilecircuit = circuit.compilecircuit(backend="pytorch", use_jdopttn=jdopttn, hyper_opt = hyper_opt, tn_simplify = False)

### Define cost function

In [ ]:
def cost(params, weight):
    results = my_compilecircuit(params)
    return weight[0]*results + weight[1] + weight[2]

### Batch executation

In [ ]:
# First dimension is the batch size
parameters = torch.rand((5, 2), requires_grad= True)
weights = torch.rand((5, 3), requires_grad= True)

In [ ]:
cost(parameters[0], weights[0])

In [ ]:
# Batch executation function
vmap_cost = vmap(cost)

In [ ]:
vmap_cost(parameters, weights)

### Hessian

In [ ]:
hess_cost = hessian(cost)

In [ ]:
hess = hess_cost(parameters[0], weights[0])
hess

### Batch executation of hessian

In [ ]:
batch_hess_cost = vmap(hess_cost)

In [ ]:
batch_hess_cost(parameters, weights)